In [1]:
import pickle
import os

file_X = 'X_data.pkl'
file_y = 'y_data.pkl'

if not os.path.exists(file_X) or not os.path.exists(file_y):
    with open(file_X, 'wb') as f:
        pickle.dump(X_master, f)
    with open(file_y, 'wb') as f:
        pickle.dump(y_master, f)
    print(f"Data has been saved to {file_X} and {file_y}")
else:
    # If the files exist, load X and y from the pickle files
    with open(file_X, 'rb') as f:
        X = pickle.load(f)
    with open(file_y, 'rb') as f:
        y = pickle.load(f)
    print("Data has been loaded from existing pickle files.")




Data has been loaded from existing pickle files.


In [3]:
# X.head()
print(y)
print(X)


     expert_consensus
0             Seizure
1             Seizure
2             Seizure
3             Seizure
4             Seizure
...               ...
5995              LPD
5996              LPD
5997            Other
5998            Other
5999            Other

[6000 rows x 1 columns]
          0           1           2           3           4           5       \
0     -80.519997  -70.540001  -80.110001 -108.750000 -120.330002  -88.620003   
1    -129.460007 -115.580002 -115.129997 -157.440002 -166.979996 -126.260002   
2     -78.440002  -91.230003  -95.889999 -122.519997  -82.949997 -101.040001   
3    -138.520004 -147.820007 -139.910004 -163.320007 -174.929993 -151.270004   
4    -100.360001 -131.580002 -123.239998 -143.029999  -92.760002  -81.739998   
...          ...         ...         ...         ...         ...         ...   
5995  432.519989  462.899994  461.119995  453.079987  459.649994  513.380005   
5996  355.970001  389.579987  452.459991  464.179993  476.750000  447.7

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import kurtosis, skew
from pywt import wavedec

# Each row represents a channel's time-series data
eeg_data = X

# Feature Extraction Functions
def extract_time_domain_features(signal):
    """Extract basic statistical features."""
    return {
        'mean': np.mean(signal),
        'variance': np.var(signal),
        'std_dev': np.std(signal),
        'skewness': skew(signal),
        'kurtosis': kurtosis(signal),
        'entropy': -np.sum(signal * np.log2(np.abs(signal) + 1e-10)),
        'zero_crossings': np.sum(np.diff(np.sign(signal)) != 0)
    }

def extract_frequency_domain_features(signal, sf=256, band_limits=[(0.5, 4), (4, 8), (8, 12), (12, 30), (30, 45)]):
    """Extract frequency features using Welch's method."""
    features = {}
    freqs, psd = welch(signal, sf, nperseg=1024)
    total_power = np.sum(psd)
    
    for i, (low, high) in enumerate(band_limits):
        band_power = np.sum(psd[(freqs >= low) & (freqs < high)])
        features[f'band_{i}_power'] = band_power
        features[f'band_{i}_relative_power'] = band_power / total_power
    
    return features

def extract_wavelet_features(signal, wavelet='db4', level=4):
    """Extract wavelet coefficients."""
    coeffs = wavedec(signal, wavelet, level=level)
    features = {}
    for i, coeff in enumerate(coeffs):
        features[f'wavelet_{i}_mean'] = np.mean(coeff)
        features[f'wavelet_{i}_variance'] = np.var(coeff)
    return features

# Process Each Channel
extracted_features = []
for index, row in eeg_data.iterrows():
    row_features = {}
    signal = row.values
    
    # Time-Domain Features
    row_features.update(extract_time_domain_features(signal))
    
    # Frequency-Domain Features
    row_features.update(extract_frequency_domain_features(signal))
    
    # Wavelet Features
    row_features.update(extract_wavelet_features(signal))
    
    extracted_features.append(row_features)

features_df = pd.DataFrame(extracted_features)

features_df.to_csv('eeg_features.csv', index=False)

print("Feature extraction completed. Features saved to 'eeg_features.csv'.")


Feature extraction completed. Features saved to 'eeg_features.csv'.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score


data = pd.read_csv('eeg_features.csv')


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # This will transform string labels to numerical values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, y_encoded, test_size=0.2, random_state=42)


# {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 13, 'n_estimators': 1537}
# Train the Random Forest Classifier
forest = RandomForestClassifier(n_estimators=1000,max_depth=10,min_samples_leaf=2,min_samples_split=13,max_features='sqrt', random_state=42)
forest.fit(X_train, y_train)

# Predict on test set
y_pred = forest.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"test Accuracy: {accuracy:.4f}")


print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))





y_pred = forest.predict(X_train)


accuracy = accuracy_score(y_train, y_pred)
print(f"Train Accuracy: {accuracy:.4f}")


print(classification_report(y_train, y_pred, target_names=label_encoder.classes_))

c:\Users\mridu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test Accuracy: 0.8342
              precision    recall  f1-score   support

         GPD       0.99      0.87      0.93       214
        GRDA       0.77      0.88      0.82       181
         LPD       0.77      0.86      0.81       188
        LRDA       0.96      0.84      0.90       223
       Other       0.82      0.71      0.76       201
     Seizure       0.72      0.84      0.78       193

    accuracy                           0.83      1200
   macro avg       0.84      0.83      0.83      1200
weighted avg       0.85      0.83      0.84      1200

Train Accuracy: 0.8965
              precision    recall  f1-score   support

         GPD       0.99      0.91      0.95       786
        GRDA       0.82      0.92      0.87       819
         LPD       0.87      0.90      0.89       812
        LRDA       0.99      0.91      0.95       777
       Other       0.93      0.83      0.87       799
     Seizure       0.82      0.91      0.86       807

    accuracy                    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

data = pd.read_csv('eeg_features.csv')

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(data)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define and Train the MLP Model ---
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64, 32),  # Three hidden layers with respective neurons
    activation='relu',                # Activation function
    solver='adam',                    # Optimization algorithm
    alpha=0.0001,                     # L2 regularization (default)
    batch_size=32,                    # Mini-batch size
    learning_rate='adaptive',         # Learning rate adjustment
    max_iter=300,                     # Maximum number of iterations
    random_state=42
)

# Fit the model
mlp.fit(X_train_scaled, y_train)

# --- Evaluate the Model ---
y_pred = mlp.predict(X_test_scaled)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("Train accuracy")
y_pred = mlp.predict(X_train_scaled)

print("Accuracy:", accuracy_score(y_train, y_pred))
print("\nClassification Report:\n", classification_report(y_train, y_pred, target_names=label_encoder.classes_))



C:\Users\mridu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.885

Classification Report:
               precision    recall  f1-score   support

         GPD       0.91      0.94      0.93       200
        GRDA       0.82      0.96      0.89       200
         LPD       0.92      0.92      0.92       200
        LRDA       0.93      0.98      0.95       200
       Other       0.87      0.70      0.78       200
     Seizure       0.87      0.80      0.84       200

    accuracy                           0.89      1200
   macro avg       0.89      0.88      0.88      1200
weighted avg       0.89      0.89      0.88      1200

Train accuracy
Accuracy: 0.9560416666666667

Classification Report:
               precision    recall  f1-score   support

         GPD       0.97      0.95      0.96       800
        GRDA       0.89      0.99      0.94       800
         LPD       0.96      0.97      0.96       800
        LRDA       0.98      0.98      0.98       800
       Other       0.98      0.89      0.93       800
     Seizure       0.9

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [(128, 64, 32), (256, 128, 64), (128, 128, 128)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'batch_size': [16, 32, 64],
    'learning_rate': ['adaptive', 'constant'],
    'max_iter': [300, 500],
}

grid_search = GridSearchCV(
    estimator=MLPClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=3,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load extracted features
data = pd.read_csv('eeg_features.csv')


imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(data)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define and Train the SVM Model ---
svm = SVC(
    kernel='rbf',             # Radial basis function kernel
    C=100,                    # Regularization parameter
    gamma=1,            # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    random_state=42           # Ensures reproducibility
)

# Fit the model
svm.fit(X_train_scaled, y_train)

print("Test Accuracy")
y_pred = svm.predict(X_test_scaled)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("Train Accuracy")
y_pred = svm.predict(X_train_scaled)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_train, y_pred))
print("\nClassification Report:\n", classification_report(y_train, y_pred, target_names=label_encoder.classes_))





C:\Users\mridu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Accuracy
Accuracy: 0.905

Classification Report:
               precision    recall  f1-score   support

         GPD       0.93      0.96      0.95       200
        GRDA       0.89      0.94      0.92       200
         LPD       0.93      0.94      0.94       200
        LRDA       0.94      0.94      0.94       200
       Other       0.82      0.81      0.82       200
     Seizure       0.93      0.82      0.87       200

    accuracy                           0.91      1200
   macro avg       0.91      0.90      0.90      1200
weighted avg       0.91      0.91      0.90      1200

Train Accuracy
Accuracy: 0.9766666666666667

Classification Report:
               precision    recall  f1-score   support

         GPD       0.99      0.97      0.98       800
        GRDA       0.94      0.97      0.95       800
         LPD       0.97      0.99      0.98       800
        LRDA       0.99      0.98      0.98       800
       Other       0.98      0.97      0.97       800
     Sei

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
svm = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100],           # Regularization parameter
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],  # Kernel coefficient
    'kernel': ['rbf', 'linear', 'poly'],  # Kernels to test
}

# --- Apply GridSearchCV ---
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    scoring='accuracy',      # Metric to optimize
    cv=5,                    # 5-fold cross-validation
    verbose=3,               # Print progress
    n_jobs=-1                # Use all processors
)

grid_search.fit(X_train_scaled, y_train)

# --- Best Parameters and Model Evaluation ---
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters:", best_params)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test_scaled)
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))




Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Best Cross-Validation Accuracy: 0.8608333333333332

Test Accuracy: 0.905

Classification Report:
               precision    recall  f1-score   support

         GPD       0.93      0.96      0.95       200
        GRDA       0.89      0.94      0.92       200
         LPD       0.93      0.94      0.94       200
        LRDA       0.94      0.94      0.94       200
       Other       0.82      0.81      0.82       200
     Seizure       0.93      0.82      0.87       200

    accuracy                           0.91      1200
   macro avg       0.91      0.90      0.90      1200
weighted avg       0.91      0.91      0.90      1200

